In [1]:
import numpy as np
import pandas as pd
from openbb import obb
obb.user.preferences.output_type = "dataframe"

In [2]:
chains = obb.derivatives.options.chains(
    "SPY",
    provider="cboe"
)

In [3]:
(
    chains
    .groupby(
        ["option_type", "strike", "expiration"]
    )
    .open_interest
    .sum()
)

option_type  strike  expiration
call         120.0   2024-12-20    104
             130.0   2024-12-20     20
             140.0   2024-12-20      3
             150.0   2024-09-20     66
                     2024-12-20     18
                                  ... 
put          840.0   2026-01-16      0
                     2026-06-18      0
                     2026-12-18      0
             845.0   2026-06-18      0
                     2026-12-18      0
Name: open_interest, Length: 9800, dtype: int64

In [4]:
#“Applying different methods to different columns”

In [5]:
(
    chains
    .groupby(
        ["option_type", "strike", "expiration"]
    )
    .agg({
        "last_trade_price": "max",
        "open_interest": "sum"
    })
)

last_trade_price  open_interest
option_type strike expiration                                 
call        120.0  2024-12-20            442.26            104
            130.0  2024-12-20            420.41             20
            140.0  2024-12-20              0.00              3
            150.0  2024-09-20            411.06             66
                   2024-12-20              0.00             18
...                                         ...            ...
put         840.0  2026-01-16              0.00              0
                   2026-06-18              0.00              0
                   2026-12-18              0.00              0
            845.0  2026-06-18              0.00              0
                   2026-12-18              0.00              0

[9800 rows x 2 columns]

In [7]:
#Applying custom functions

In [8]:
(
    chains
    .groupby(["option_type"])
    .apply(lambda x: (x["ask"] - x["bid"]).mean(),
           include_groups=False)
)

option_type
call    0.980753
put     0.517578
dtype: float64

In [10]:
# Grouping and transforming data
'''
transform: Used after groupby to return a Series with the same shape as the original DataFrame.
Replaces: Each group with the value from a method applied to that group.
Example: Compute the z-score of last_trade_price within each expiration date group.
Result: A Series of the same length as chains, where last_trade_price is replaced with its z-score value within each expiration date group.
Use case: The z-score puts all the prices on the same scale relative to their specific group, enabling comparisons.
'''